In [3]:
# Load the cc12m file

import csv
from tqdm import tqdm

cc12m_dict = {}

with open("cc12m.tsv") as file:
    tsv_file = csv.reader(file, delimiter="\t")

    for i, line in tqdm(enumerate(tsv_file)):
        cc12m_dict[i] = line

12423374it [00:39, 313194.28it/s]


In [4]:
num_processes = 100
num_samples_per_process = 100

width = 10000
start_idx = 0
end_idx = start_idx + (num_processes * num_samples_per_process - 1)

In [5]:
print (start_idx)
print (end_idx)

0
9999


In [6]:
import requests
import multiprocessing as mp
from multiprocessing import Process, Manager


def download_one_record(record_meta_data, record_id, res_dict):
    '''
    record_meta_data: contains the link and the text description of the image for the link
    record_id: record id will be the image id of the stored image
    '''
    
    link = record_meta_data[0]
    image_desc = record_meta_data[1]
    
    try:
        img_data = requests.get(link, timeout=10).content
        with open(f'image_{record_id}.jpg', 'wb') as handler:
            handler.write(img_data)
        res_dict[record_id] = 1
    except:
        res_dict[record_id] = 0
    

def download_records_1_process(meta_dict, start_idx, num_records, res_dict):
    '''
    meta_dict: dictionary of meta data
    start_idx: start index of download
    num_records: number of records to be downloaded
    res_dict: result dictionary containing information of how many records were downloaded
    '''
    
    for idx in range(start_idx, start_idx + num_records):
        if idx in meta_dict.keys():
            record_meta_data = meta_dict[idx]
            download_one_record(record_meta_data, idx, res_dict)


manager = Manager()
shared_res_dict = manager.dict()

def download_records_multi_process(meta_dict, start_idx, num_records_per_process, num_processes, res_dict):
    '''
    meta_dict: dictionary of meta data
    start_idx: start index of download for all processes
    num_records_per_process: number of records to be downloaded per process
    num_processes: number of processes to be launched
    res_dict: result dictionary containing information of which records were stored correctly
    '''
    moving_idx = start_idx
    proc_list = []
    
    print ('Starting processes...')
    for p in tqdm(range(num_processes)):
        proc = Process(target=download_records_1_process, args=(meta_dict, moving_idx, num_records_per_process, res_dict))
        moving_idx = moving_idx + num_records_per_process
        proc.start()
        proc_list.append(proc)
    
    print ('Joining processes...')
    for proc in tqdm(proc_list):
        proc.join()

In [7]:
download_records_multi_process(cc12m_dict, 0, 100, 10, shared_res_dict)

Starting processes...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16.77it/s]


Joining processes...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:57<00:00, 29.72s/it]


In [10]:
for k in shared_res_dict.keys():
    if shared_res_dict[k] == 0:
        print (k)

101
120
422
16
17
918
123
923
925
847
30
739
336
227
652
932
855
562
449
42
346
663
665
572
457
458
239
773
780
887
889
975
259
491
87
983
399
88
274
285
